# Output Parsing

This notebook covers functionality related to output parsing. For more information, see:

- [List of all Output Parsers](https://js.langchain.com/docs/modules/model_io/output_parsers/types/)
- [OpenAI Function Calling](https://js.langchain.com/docs/modules/model_io/chat/function_calling)

## Converting Messages

In [4]:
import { ChatOpenAI } from "@langchain/openai";
import { ChatPromptTemplate } from "@langchain/core/prompts";

In [5]:
const prompt = ChatPromptTemplate.fromTemplate("Tell me a joke about {topic}");
const model = new ChatOpenAI({
  modelName: "gpt-3.5-turbo",
  temperature: 0,
});

In [6]:
const chain = prompt.pipe(model);

In [7]:
await chain.invoke({ topic: "bears" });

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Why don't bears wear shoes?\n\nBecause they have bear feet!",
    additional_kwargs: { function_call: undefined, tool_calls: undefined }
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Why don't bears wear shoes?\n\nBecause they have bear feet!",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined }
}

In [8]:
import { StringOutputParser } from "@langchain/core/output_parsers";

const parser = new StringOutputParser();
const chain = prompt.pipe(model).pipe(parser);

In [9]:
await chain.invoke({ topic: "bears" });

"Why don't bears wear shoes?\n\nBecause they have bear feet!"

## OpenAI Function Calling

In [10]:
const openAIFunctions = [
  {
    name: "joke_teller",
    description: "A joke to tell to the user.",
    parameters: {
      type: "object",
      properties: {
        setup: {
          type: "string",
          description: "question to set up a joke",
        },
        punchline: {
          type: "string",
          description: "answer to resolve the joke",
        },
      },
      required: ["setup", "punchline"],
    },
  }
]

In [11]:
import { JsonOutputFunctionsParser } from "langchain/output_parsers";


In [12]:
const parser = new JsonOutputFunctionsParser<{ setup: string, punchline: string }>();

In [13]:
const chain = prompt.pipe(model.bind({ functions: openAIFunctions })).pipe(parser);

In [14]:
await chain.invoke({ topic: "bears" });

{
  setup: "Why don't bears wear shoes?",
  punchline: "Because they have bear feet!"
}

## Streaming

In [15]:
for await (const step of await chain.stream({
  topic: "bears",
})) {
  console.log(step);
}

{}
{ setup: "" }
{ setup: "Why" }
{ setup: "Why don" }
{ setup: "Why don't" }
{ setup: "Why don't bears" }
{ setup: "Why don't bears wear" }
{ setup: "Why don't bears wear shoes" }
{ setup: "Why don't bears wear shoes?", punchline: "" }
{ setup: "Why don't bears wear shoes?", punchline: "Because" }
{ setup: "Why don't bears wear shoes?", punchline: "Because they" }
{
  setup: "Why don't bears wear shoes?",
  punchline: "Because they have"
}
{
  setup: "Why don't bears wear shoes?",
  punchline: "Because they have bear"
}
{
  setup: "Why don't bears wear shoes?",
  punchline: "Because they have bear feet"
}
{
  setup: "Why don't bears wear shoes?",
  punchline: "Because they have bear feet!"
}
